<a href="https://colab.research.google.com/github/luuleitner/xMason/blob/main/xmasonsim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# xMason can be directly installed from the GitHub repository using PIP.

!pip install git+https://github.com/luuleitner/xMason

In [ ]:
# We need to import necessary packages into Colab. Don't worry to much about this cell of code

from simulation.src.data.loader import VNA_Dataloader
from simulation.src.models.simulate import simulate_xMason
from simulation.src.features.ports import frequency_spectrum

import os
import numpy as np # array handeling
from urllib.request import urlretrieve # for downloading the sample data
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.signal import find_peaks

In [ ]:
mason_path = '/content/xMasonData'
mat_path = os.path.join(mason_path, 'materials.csv')

if os.path.exists(mason_path) == False:
  os.mkdir(mason_path)
  os.chdir(mason_path)
  !wget https://raw.githubusercontent.com/luuleitner/xMason/main/data/materials.csv

if os.path.exists(os.path.join(mason_path,'/ISAFdata')) == False:
  os.mkdir(os.path.join(mason_path,'/ISAFdata'))
  os.chdir(os.path.join(mason_path,'/ISAFdata'))
  !wget -i https://raw.githubusercontent.com/luuleitner/xMason/main/data/dataset_ISAF22/isaf_dataset_url.txt

os.chdir(mason_path)

In [ ]:
frequency_band = [20, 100]
transducer_radius = 0.0088
transducer_thickness = 13.7 * 10 ** -6,
electrode_thickness = 0.5 * 10 ** -6,
substrate_thickness = 13.7 * 10 ** -6,


Simulation_1 = simulate_xMason(parameters={'fband' : frequency_band,
                                           'radius' : transducer_radius,
                                           'thickness_td' : transducer_thickness,
                                           'thickness_el' : electrode_thickness,
                                           'thickness_sub' : substrate_thickness,
                                           'Tload' : 'Air',
                                           'Telectrode' : 'Silverink',
                                           'piezo' : 'P(VDF-TrFE)',
                                           'Belectrode' : 'Silverink',
                                           'Bsubstrate' : 'Kapton',
                                           'Bload' : 'Air'},
                               matpath=mat_path)